In [1]:
import pandas as pd
# Read dataframe
malConll = pd.read_csv('malExpanded.conllu')
malConll['word'] = malConll['word'].astype(str)
malConll['lemma'] = malConll['lemma'].astype(str)
malConll['DEPS'] = malConll['DEPS'].astype(str)
#Convert all deps to lowecase
malConll['DEPS'] = malConll['DEPS'].str.lower()


malConll.loc[malConll['word']=='ഉള്ള','lemma'] = 'ഉള്'

repDict = {'k1' : ['k1*u','k1a','k1s','k1u','K1U','K1u','ras-k1'], 
           'k2':['k2g','k2p','k2s','k2u','K2U','K2u'], 
           'k4':['k4a','k4u','rask4'],
          'k7':['k7a','k7p','k7t'],
          
           'vmod':['vmod-emph', 'vmod_emph', 'vmodemph', 'vmod-relc', 'vmod-emp', 'vmod_relc',
                   'vmod-enmph', 'vmode-emph','vnmod-emph','vmood-emph'],
        'nmod':[ 'nmod-emph', 'vnmod-emph', 'nmod__k1inv', 'nmod-relc', 'nmod-emp','nmod__relc']
          }

for item in repDict:
    for stuff in repDict[item]:
        malConll.loc[malConll['DEPS']==stuff,'DEPS'] = item
malConll.groupby('DEPS').count().sort_values('sentID',ascending=False)

/home/ashim/indicmt/env_mt/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,sentID,wordPosition,word,lemma,UPOS,XPOS,FEATS,HEAD,DEPREL,MISC,category,gender,number,person,case,vib,tam
DEPS,,,,,,,,,,,,,,,,,,
k1,26465,26465,26465,26465,26465,26462,26465,26465,26465,26465,26465,26031,19071,18842,18797,18791,18463,18774
root,14063,14063,14063,14063,14063,14058,14063,14063,14063,14063,14063,13332,60,27,26,25,6422,6657
k7,13998,13998,13998,13998,13998,13997,13998,13998,13998,13998,13998,13855,9757,9724,9720,9707,9591,9789
rsym,13277,13277,13277,13277,13277,13271,13277,13277,13277,13277,13277,13193,40,18,18,18,29,29
k2,11249,11249,11249,11249,11249,11245,11249,11249,11249,11249,11249,11123,8023,7976,7967,7964,7865,7992
nmod,11245,11245,11245,11245,11245,11244,11245,11245,11245,11245,11245,11069,2659,2615,2615,2612,6851,10117
vmod,10441,10441,10441,10441,10441,10439,10441,10441,10441,10441,10441,10168,289,235,232,233,6448,9321
r6,8925,8925,8925,8925,8925,8925,8925,8925,8925,8925,8925,8851,7330,7324,7310,7309,7402,7436
k4,5457,5457,5457,5457,5457,5456,5457,5457,5457,5457,5457,5399,4026,4016,4012,4011,4082,4090


In [2]:
# malConll[malConll['word'].str.len() <= 1]['word'].unique()
wg = malConll.groupby('DEPS').count().sort_values('word')
wgi = (wg[wg['lemma'] < 400]['lemma'])
sent_ids_remove = malConll[malConll['DEPS'].isin(wgi.index)]['sentID'].unique()
df = malConll[~malConll['sentID'].isin(sent_ids_remove)]
df.groupby('DEPS').count().sort_values('sentID',ascending=False)

,Unnamed: 0,sentID,wordPosition,word,lemma,UPOS,XPOS,FEATS,HEAD,DEPREL,MISC,category,gender,number,person,case,vib,tam
DEPS,,,,,,,,,,,,,,,,,,
k1,25937,25937,25937,25937,25937,25934,25937,25937,25937,25937,25937,25508,18674,18447,18402,18399,18071,18379
root,13773,13773,13773,13773,13773,13768,13773,13773,13773,13773,13773,13058,59,26,25,24,6288,6517
k7,13728,13728,13728,13728,13728,13727,13728,13728,13728,13728,13728,13589,9578,9545,9542,9528,9417,9594
rsym,13018,13018,13018,13018,13018,13012,13018,13018,13018,13018,13018,12937,37,18,18,18,29,29
k2,11026,11026,11026,11026,11026,11022,11026,11026,11026,11026,11026,10905,7854,7810,7801,7798,7705,7826
nmod,10972,10972,10972,10972,10972,10971,10972,10972,10972,10972,10972,10801,2602,2562,2562,2559,6684,9869
vmod,10232,10232,10232,10232,10232,10230,10232,10232,10232,10232,10232,9964,276,224,221,222,6312,9133
r6,8747,8747,8747,8747,8747,8747,8747,8747,8747,8747,8747,8673,7175,7169,7158,7157,7245,7279
k4,5380,5380,5380,5380,5380,5379,5380,5380,5380,5380,5380,5322,3962,3952,3948,3947,4018,4026


In [3]:
# Remove Punctuation words
regex_punc_str = '[\x00-\x2F\x3A-\x40\x5B-\x60-\x7B-\x7F]'
df['word'] = df['word'].str.replace(regex_punc_str, '', regex=True)
df['lemma'] = df['lemma'].str.replace(regex_punc_str, '', regex=True)
punc_remove_index = df[(df['word'] == '') | (df['lemma'] == '')].index
df_new = df[~(df.index.isin(punc_remove_index))]
remove_upos = ['RD__PUNC', 'RD_SYM', 'RD_PUNC','RD__SYM']
remove_upos_index = df_new[df_new['UPOS'].isin(remove_upos)].index
df_new = df_new[~(df_new.index.isin(remove_upos_index))]


# Remove sentences with nan at word or lemma
word_lemma_nan_sent_ids = df_new[(df_new['lemma'].str.lower() == 'nan') | (df_new['word'].str.lower() == 'nan')]['sentID'].unique()
df_new = df_new[~(df_new['sentID'].isin(word_lemma_nan_sent_ids))]
df_new['sentID'].unique().shape

# df_new[(df_new['word'] == '') | (df_new['lemma'] == '')]['sentID'].unique().shape == (0,)
# df_new[df_new['word'].str.len() <= 1]['word'].unique()

/home/ashim/indicmt/env_mt/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/ashim/indicmt/env_mt/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


(11845,)

In [ ]:
# Save 10 examples of each dependency type 

# deps = ['k1', 'k5', 'k7', 'k7p', 'k4', 'k1s', 'k2', 'k7t', 'k2s', 'k1u', 'k3', 'k2u', 'K1U', 'k2p', 'k4a', 'K1u', 'k1*u', 'kiu', 'k7a', 'k4u', 'K2U', 'k2g', 'K2u', 'k6',  'k1a']
# group_deps = df_new.groupby('DEPS')
# deps_sentes = {}
# for dep in deps:
#     deps_sentes[dep] = []
# for dep in deps:
#     count = 10
#     gp_index = group_deps.groups[dep]
#     sent_ids = list(df_new[df_new.index.isin(gp_index)]['sentID'].unique())
#     if len(sent_ids) < 10:
#         deps_sentes[dep] = sent_ids
#     else:
#         deps_sentes[dep] = sent_ids[:10]
# s = ",".join(["None" for i in range(df_new.shape[1]-1)])
# with open('dependency_examples.csv', 'w') as fp:
#     fp.write("None,"+ s + "\n")
# with open('dependency_examples.csv', 'a') as fp:
#     for dep in deps:
#         fp.write(str(dep) + "," + s + "\n")
#         df_new.to_csv(fp, header=False)
#     fp.write("\n\n")

In [13]:
df_new[df_new['word'].str.len() <= 1]['word'].unique()
sentToremove = df[df['Unnamed: 0']==7489]['sentID'].unique()
df_new = df_new[~(df_new['sentID'].isin(sentToremove))]
df_new['sentID'].unique().shape

(11844,)

In [14]:
import importlib
import wordpairalignment #import the module here, so that it can be reloaded.
importlib.reload(wordpairalignment)

<module 'wordpairalignment' from '/home/ashim/indicmt/wordpairalignment.py'>

In [26]:
df = df_new
df = df.head(50000)
df['source'] = df['lemma']
df['target'] = df['word']
df_align= df.apply(wordpairalignment.iterLCS, axis=1)
df_align['extracted'] = df_align['aligned'].str.extract('INS\((.+?)\)')
df_align.shape

/home/ashim/indicmt/env_mt/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/ashim/indicmt/env_mt/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


SW1 ഇന്രെര്ഫെറോന്
SW2 ഇന്രെര്ഫെറോന്
common ഇന്രെര്ഫെറോന്
ordermatchilla ['1', '൯'] ['1']
ordermatchilla ['1', '൯'] ['1']
ordermatchilla ['1', '൯'] ['1']
ordermatchilla ['1', '൯'] ['1']
ordermatchilla ['1', '൯'] ['1']
ordermatchilla ['2'] []
ordermatchilla ['1', '2', '൯'] ['1', '2']
ordermatchilla ['1', '൯'] ['1']
ordermatchilla ['1', '൯'] ['1']
ordermatchilla ['1', '൯'] ['1']
ordermatchilla ['1', '൯'] ['1']
ordermatchilla ['1', '൯'] ['1']
ordermatchilla ['1', '൯'] ['1']
ordermatchilla ['1', '2', '3', '4'] ['1', '4', '3', '2']
SW2 നേര്ത്ത്നേര്ത്ത്
common നേര്ത്ത്നേര്ത്ത്
ordermatchilla ['2', '3', '1'] ['2', '1', '3']
SW2 താത്പര്യ
common താത്പര്യ
ordermatchilla ['2', '1', '3'] ['2', '3', '1']
SW2 ആസ്രരമെഡിസിര്രി
common ആസ്രരമെഡിസിര്രി
here സര മത്സര മല്‍സരത്തില്‍
ordermatchilla ['1'] ['1', '27']
ordermatchilla ['57'] []
here ര് നിഷ്ക്കര്ഷി നിഷ്‌കര്‍ഷിക്കുന്നത്
here ഷി നിഷ്ക്കര്ഷി നിഷ്‌കര്‍ഷിക്കുന്നത്
ordermatchilla ['1', '2'] ['1', '47', '2']
ordermatchilla ['1', '2', '3'] ['1', '3', '2']

(50000, 28)

In [ ]:
df_align[df['vib'] == '']